In [1]:
import numpy as np
import pandas as pd
import os
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import cmath
import math

cwd = os.getcwd()
sys.path.append(cwd+"/../handy_functions")

from concat_pickles import get_outputs,get_outputsSweep,print_df
from scipy.stats import mannwhitneyu, normaltest, zscore
from statannotations.Annotator import Annotator

pal = sns.husl_palette(8)
mtfef_pal = ['forestgreen','mediumpurple']

data_folder = '/Users/kendranoneman/Projects/mayo/NeuralDecoding/runs'
fig_folder = '/Users/kendranoneman/Projects/mayo/NeuralDecoding/figures/grc'

In [ ]:
def rtoZ(r2):
    z = 0.5 * math.log((1+r2)/(1-r2))
    return z

In [ ]:
conditions = ["bn0"]
column_names = ['sess','repeat','outer_fold','nMT','nFEF','model','mean_R2','mean_rho','mean_R2_null','mean_rho_null']
df = get_outputs(data_folder+'/all_decoders',conditions,column_names)
df.head(1)

# FIGURE 2

In [ ]:
plt.rcParams.update({'font.size': 12})

sess = 29
wf = (df.loc[(df['model']==0) & (df['sess']==sess)]['mean_R2']).values
wc = (df.loc[(df['model']==1) & (df['sess']==sess)]['mean_R2']).values
xgb = (df.loc[(df['model']==2) & (df['sess']==sess)]['mean_R2']).values
svr = (df.loc[(df['model']==3) & (df['sess']==sess)]['mean_R2']).values
dnn = (df.loc[(df['model']==4) & (df['sess']==sess)]['mean_R2']).values
rnn = (df.loc[(df['model']==5) & (df['sess']==sess)]['mean_R2']).values
gru = (df.loc[(df['model']==6) & (df['sess']==sess)]['mean_R2']).values
lstm = (df.loc[(df['model']==7) & (df['sess']==sess)]['mean_R2']).values

n=10
means=([np.mean(wf),np.mean(wc),np.mean(xgb),np.mean(svr),np.mean(dnn),np.mean(rnn),np.mean(gru),np.mean(lstm)])
err=([np.std(wf)*np.sqrt(1./n+1./(n-1)),np.std(wc)*np.sqrt(1./n+1./(n-1)),np.std(xgb)*np.sqrt(1./n+1./(n-1)),np.std(svr)*np.sqrt(1./n+1./(n-1)),np.std(dnn)*np.sqrt(1./n+1./(n-1)),np.std(rnn)*np.sqrt(1./n+1./(n-1)),np.std(gru)*np.sqrt(1./n+1./(n-1)),np.std(lstm)*np.sqrt(1./n+1./(n-1))])


fig, ax = plt.subplots(1,1,figsize=(8,4))

ind = np.arange(len(err)) #X values for plotting
for pos, y, yerr, color in zip(ind, means, err, pal): #Loop through methods and plot error bars
    ax.errorbar(pos, y, yerr, lw=3, capsize=5, capthick=2, color=color)

    #Remove x tick labels
    ax.set_xticklabels(['','wf','wc','xgb','svr','dnn','rnn','gru','lstm'], rotation=0)

    #Remove right and top boundaries, and make ticks inward
    ax.tick_params(direction='in',bottom=0)
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')

    #Plot individual R2 values for each fold as an 'x'
    scatter_x=np.reshape(np.transpose(np.ones((10,1))*range(8)),(80,1)) #Get x values for plotting (first 10 will have an x value of 0, second 10 will have an x value of 1, etc)
    scatter_y=np.concatenate((wf,wc,xgb,svr,dnn,rnn,gru,lstm),axis=0) #Y values for plotting
    colors_list=[] #Create a list of the colors that should be used when plotting each 'x'
    for i in scatter_x.astype(int).reshape((1,-1))[0]:
        colors_list.append(pal[i])
        
    ax.scatter(scatter_x,scatter_y,c=colors_list,marker='x',alpha=0.05)
fig.savefig(fig_folder+'/model_R2.png', dpi=600, bbox_inches='tight')

In [ ]:
with open(data_folder+'/all_decoders/fold06-m6-eyetrace.pickle', 'rb') as f:
    y_test,y_test_predicted = pickle.load(f)
    
#plt.rcParams.update({'font.size': 14})
x = np.arange(0, 31*50, 50) - 500
print(x)
fig, axes = plt.subplots(2,2,figsize=(5,3))
fig.tight_layout()

start_end = [1272,1303]
axes[0,0].plot(x,y_test[start_end[0]:start_end[1],0], color=(0.3,0.3,0.3), linewidth=2, label='true (horizontal)')
axes[0,0].plot(x,y_test_predicted[start_end[0]:start_end[1],0], color=pal[6], linewidth=2, label='predicted (horizontal)')
axes[0,0].plot(x,y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linestyle='dashed', linewidth=2, label='true (vertical)')
axes[0,0].plot(x,y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linestyle='dashed', linewidth=2, label='predicted (vertical)')
axes[0,0].set_ylim(-12.5, 14.5)
axes[0,0].set_yticks([-10,0,10])
axes[0,0].set_xticks([])

start_end = [87,118] 
axes[0,1].plot(x,y_test[start_end[0]:start_end[1],0], color=(0.3,0.3,0.3), linewidth=2, label='$X_{true}$')
axes[0,1].plot(x,y_test_predicted[start_end[0]:start_end[1],0], color=pal[6], linewidth=2, label='$X_{decoded}$')
axes[0,1].plot(x,y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linestyle='dashed', linewidth=2, label='$Y_{true}$')
axes[0,1].plot(x,y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linestyle='dashed', linewidth=2, label='$Y_{decoded}$')
axes[0,1].set_ylim(-1.25, 12.5)
axes[0,1].set_yticks([0,10])
axes[0,1].set_xticks([])

start_end = [1047,1078] 
axes[1,0].plot(x,y_test[start_end[0]:start_end[1],0], color=(0.3,0.3,0.3), linewidth=2, label='true (horizontal)')
axes[1,0].plot(x,y_test_predicted[start_end[0]:start_end[1],0], color=pal[6], linewidth=2, label='predicted (horizontal)')
axes[1,0].plot(x,y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linestyle='dashed', linewidth=2, label='true (vertical)')
axes[1,0].plot(x,y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linestyle='dashed', linewidth=2, label='predicted (vertical)')
axes[1,0].set_ylim(-12.5,2)
axes[1,0].set_yticks([-10,0])
axes[1,0].set_xticks([-500,0,500,1000])

start_end = [184,215] 
axes[1,1].plot(x,y_test[start_end[0]:start_end[1],0], color=(0.3,0.3,0.3), linewidth=2, label='true (horizontal)')
axes[1,1].plot(x,y_test_predicted[start_end[0]:start_end[1],0], color=pal[6], linewidth=2, label='predicted (horizontal)')
axes[1,1].plot(x,y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linestyle='dashed', linewidth=2, label='true (vertical)')
axes[1,1].plot(x,y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linestyle='dashed', linewidth=2, label='predicted (vertical)')
axes[1,1].set_ylim(-14.75, 14)
axes[1,1].set_yticks([-10,0,10])
axes[1,1].set_xticks([-500,0,500,1000])

plt.subplots_adjust(wspace=0.25, hspace=0.05)

axes[0,1].legend(loc='lower center', bbox_to_anchor=(-0.12, 0.95),
          fancybox=False, shadow=False, ncol=4, fontsize="10",frameon=False)

#fig.savefig(fig_folder+'/model_4traces.png', dpi=600, bbox_inches='tight') 

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(3,3))

start_end = [1272,1303]
axes.plot(y_test[start_end[0]:start_end[1],0],y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linewidth=2, label='true (horizontal)')
axes.plot(y_test_predicted[start_end[0]:start_end[1],0],y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linewidth=2, label='predicted (horizontal)')

start_end = [87,118] 
axes.plot(y_test[start_end[0]:start_end[1],0],y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linewidth=2, label='true (horizontal)')
axes.plot(y_test_predicted[start_end[0]:start_end[1],0],y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linewidth=2, label='predicted (horizontal)')

start_end = [1047,1078] 
axes.plot(y_test[start_end[0]:start_end[1],0],y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linewidth=2, label='true (horizontal)')
axes.plot(y_test_predicted[start_end[0]:start_end[1],0],y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linewidth=2, label='predicted (horizontal)')

start_end = [184,215] 
axes.plot(y_test[start_end[0]:start_end[1],0],y_test[start_end[0]:start_end[1],1], color=(0.3,0.3,0.3), linewidth=2, label='true (horizontal)')
axes.plot(y_test_predicted[start_end[0]:start_end[1],0],y_test_predicted[start_end[0]:start_end[1],1], color=pal[6], linewidth=2, label='predicted (horizontal)')

#fig.savefig(fig_folder+'/model_stacktraces.png', dpi=600, bbox_inches='tight') 

# Figure 3

In [ ]:
conditions = ["m00"]
column_names = ['sess','repeat','outer_fold','nMT','nFEF','model','mean_R2','mean_rho','mean_R2_null','mean_rho_null','time_elapsed','neurons']

df2 = get_outputsSweep(data_folder+'/neuron_sweep',conditions,column_names)
df2.head(1)

In [ ]:
plt.rcParams.update({'font.size': 12})

fig, ax = plt.subplots(1,1, figsize=(5,5))
fig.tight_layout(pad=2)

df = df2.loc[df2['sess']==29] #16,19,20,24 
df = df[['nMT','nFEF','mean_R2']]
df_m = df.groupby(['nFEF', 'nMT'])["mean_R2"].mean().reset_index(name="R2")
result = df_m.pivot(index='nFEF', columns='nMT', values='R2')

print(result)

#YlGnBu

a = sns.heatmap(ax=ax,data=result,cmap="Blues", mask=result.isnull(), alpha=0.6, xticklabels=2,yticklabels=2, cbar_kws={'location':'right','ticks':[0.1,0.2,0.3,0.4,0.5,0.6],'orientation':'vertical','pad':0.01,"shrink":0.9})
a.invert_yaxis()
plt.yticks(rotation=0) 
a.set(xlabel=None)
a.set(ylabel=None)

#fig.savefig(fig_folder+'/r2_numNeurons-heatmap.png', dpi=300, bbox_inches='tight') 

In [ ]:
plt.rcParams.update({'font.size': 14})

df = df2[['nMT','nFEF','mean_R2','sess','repeat','outer_fold']]
df = df.loc[df2['sess']==29]

conditions = [
    (df['nFEF'] == 0),
    (df['nMT'] == 0),
    (df['nMT'] == df['nFEF']),
    (df['nMT'] > 0) & (df['nFEF'] > 0) & (df['nMT'] != df['nFEF'])
    ]

values = ['mt only', 'fef only', 'mt=fef','mt+fef']
df['condition'] = np.select(conditions, values)
df['num_neurons'] = df.loc[:,['nMT','nFEF']].sum(axis=1)
df = df.loc[(df['num_neurons']<=np.array([df['nMT'].max(),df['nFEF'].max()]).max())]

df_mn = df.groupby(['nFEF', 'nMT', 'num_neurons'])["mean_R2"].mean().reset_index(name="mR2")
df_sd = df.groupby(['nFEF', 'nMT', 'num_neurons'])["mean_R2"].std().reset_index(name="sR2")

fef_mn = df_mn.loc[df_mn['nMT']==0]['mR2'].values
fef_sd = (df_sd.loc[df_sd['nMT']==0]['sR2'].values)*np.sqrt(1./10+1./(10-1))
mt_mn = df_mn.loc[df_mn['nFEF']==0]['mR2'].values
mt_sd = (df_sd.loc[df_sd['nFEF']==0]['sR2'].values)*np.sqrt(1./10+1./(10-1))
mf_mn = df_mn.loc[df_mn['nFEF']==df_mn['nMT']]['mR2'].values
mf_sd = (df_sd.loc[df_sd['nFEF']==df_mn['nMT']]['sR2'].values)*np.sqrt(1./10+1./(10-1))

fig, ax = plt.subplots(1,1,figsize=(5,6))

ax.errorbar(np.arange(2,2+(len(fef_mn)*2),2), fef_mn, fef_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color='mediumpurple', zorder=2)
ax.errorbar(np.arange(2,2+(len(mt_mn)*2),2), mt_mn, mt_sd, lw=3, capsize=4, marker='o', alpha=0.9, markersize=7, capthick=2, color='mediumseagreen',zorder=3)
ax.errorbar(np.arange(4,4+(len(mf_mn)*4),4), mf_mn, mf_sd, lw=3, capsize=4, marker='o', alpha=0.8, markersize=7, capthick=2,  color='cornflowerblue',zorder=1)

ax.set_xticks(np.arange(0,41,4))
ax.set_yticks([0.1,0.2,0.3,0.4,0.5,0.6])
ax.set_ylim(0.01,0.678)

fig.savefig(fig_folder+'/r2_numNeurons-onesess.png', dpi=600, bbox_inches='tight') 

In [ ]:
plt.rcParams.update({'font.size': 12})

df = df2[['nMT','nFEF','mean_R2','sess','repeat','outer_fold']]
df = df.loc[df['sess']!=16]
df['zscore_R2'] =  df.groupby(['sess','outer_fold']).mean_R2.transform(lambda x : zscore(x,ddof=1))

conditions = [
    (df['nFEF'] == 0),
    (df['nMT'] == 0),
    (df['nMT'] == df['nFEF']),
    (df['nMT'] > 0) & (df['nFEF'] > 0) & (df['nMT'] != df['nFEF'])
    ]

values = ['mt only', 'fef only', 'mt=fef','mt+fef']
df['condition'] = np.select(conditions, values)
df['num_neurons'] = df.loc[:,['nMT','nFEF']].sum(axis=1)
#df = df.loc[(df['num_neurons']<=np.array([df['nMT'].max(),df['nFEF'].max()]).max())]

df_mn = df.groupby(['nFEF', 'nMT', 'num_neurons'])["mean_R2"].mean().reset_index(name="mR2")
df_sd = df.groupby(['nFEF', 'nMT', 'num_neurons'])["mean_R2"].std().reset_index(name="sR2")

fef_mn = df_mn.loc[df_mn['nMT']==0]['mR2'].values
fef_sd = (df_sd.loc[df_sd['nMT']==0]['sR2'].values)*np.sqrt(1./10+1./(10-1))
mt_mn = df_mn.loc[df_mn['nFEF']==0]['mR2'].values
mt_sd = (df_sd.loc[df_sd['nFEF']==0]['sR2'].values)*np.sqrt(1./10+1./(10-1))
mf_mn = df_mn.loc[df_mn['nFEF']==df_mn['nMT']]['mR2'].values
mf_sd = (df_sd.loc[df_sd['nFEF']==df_mn['nMT']]['sR2'].values)*np.sqrt(1./10+1./(10-1))

fig, ax = plt.subplots(1,1,figsize=(10,4))

ax.errorbar(np.arange(2,2+(len(fef_mn)*2),2), fef_mn, fef_sd, lw=3, capsize=4, marker='o', markersize=7, capthick=2, alpha=0.9,color='mediumpurple', zorder=2)
ax.errorbar(np.arange(2,2+(len(mt_mn)*2),2), mt_mn, mt_sd, lw=3, capsize=4, marker='o', markersize=7, capthick=2, alpha=0.9,color='mediumseagreen',zorder=3)
ax.errorbar(np.arange(4,4+(len(mf_mn)*4),4), mf_mn, mf_sd, lw=3, capsize=4, marker='o', markersize=7, capthick=2, alpha=0.8, color='cornflowerblue',zorder=1)

ax.set_xticks(np.arange(0,51,4))
ax.set_yticks([0,0.1,0.2,0.3,0.4,0.5,0.6])
ax.set_ylim(-0.07,0.67)

df_c = df.groupby(['condition','num_neurons','sess']).mean().reset_index()
df_c = df_c.groupby(['condition','num_neurons']).count().reset_index()
df_cnts = [df_c.loc[df_c['condition'] == 'fef only']['repeat'].values,df_c.loc[df_c['condition'] == 'mt only']['repeat'].values,df_c.loc[df_c['condition'] == 'mt=fef']['repeat'].values]

fef_ct = df_c.loc[df_c['condition']=='fef only']['sess'].values
mt_ct = df_c.loc[df_c['condition']=='mt only']['sess'].values
mf_ct = df_c.loc[df_c['condition']=='mt=fef']['sess'].values

for i, size in enumerate(fef_ct):
    x = np.arange(2,2+(len(fef_mn)*2),2)
    if i==0:
        plt.text(x[i], fef_mn[i] + fef_sd[i] + 0.01, 'N=\n'+str(size), ha='center', va='bottom', color='mediumpurple', fontsize=13)
    else:
        plt.text(x[i], fef_mn[i] + fef_sd[i] + 0.01, str(size), ha='center', va='bottom', color='mediumpurple')

for i, size in enumerate(mt_ct):
    x = np.arange(2,2+(len(mt_mn)*2),2)
    plt.text(x[i]+0.6, mt_mn[i] - mt_sd[i] - 0.05, str(size), ha='center', va='bottom', color='mediumseagreen')
    
for i, size in enumerate(mf_ct):
    x = np.arange(4,4+(len(mf_mn)*4),4)
    if x[i]>25:
        plt.text(x[i], mf_mn[i] - mf_sd[i] - 0.06, str(size), ha='center', va='bottom', color='cornflowerblue')


fig.savefig(fig_folder+'/r2_numNeurons-allsess.png', dpi=600, bbox_inches='tight') 

# Figure 4

In [ ]:
conditions = ["m00"]
column_names = ['sess','outer_fold','nMT','nFEF','model','mean_R2','mean_rho','mean_R2_null','mean_rho_null','time_elapsed','neuron_dropped','mean_R2_    FULL','mean_rho_FULL','mean_R2_null_FULL','mean_rho_null_FULL']
df2 = get_outputsSweep(data_folder+'/neuron_dropping',conditions,column_names)
df2.head(1)

sessions = df2['sess'].unique()

units = pd.DataFrame()
for s in sessions:
    u = pd.read_csv(cwd+'/../datasets/units/units-pa{:0>2d}dir4A-pre500-post300.csv'.format(s))
    units = pd.concat([units,u],ignore_index=True)
    
units['sess'] = units['Session'].str.slice(start=2, stop=4)
units.head(1)

In [ ]:
plt.rcParams.update({'font.size': 12})

sess = 29

df = df2.loc[(df2['sess']==sess)]
means = df.groupby(['neuron_dropped'])["mean_R2"].mean().reset_index(name="mR2")['mR2'].values
err = df.groupby(['neuron_dropped'])["mean_R2"].std().reset_index(name="sR2")['sR2'].values*np.sqrt(1./10+1./(10-1))

fig, ax = plt.subplots(1,1,figsize=(10,3))
fig.tight_layout()

ind = np.arange(len(err)) #X values for plotting
mtfef_pal = []
for i in range(int(df.iloc[0]['nFEF'])+1):
    if int(df.iloc[0]['nFEF']) !=0:
        mtfef_pal.append('mediumpurple')
for i in range(int(df.iloc[0]['nMT'])):
    mtfef_pal.append('mediumseagreen')
    
for pos, y, yerr, color in zip(ind, means, err, mtfef_pal): #Loop through methods and plot error bars
    ax.errorbar(pos, y, yerr, fmt='o', markersize=4,lw=2, capsize=3, capthick=2, color=color,zorder=1)
    ax.axhline((df['mean_R2_    FULL'].values).mean(),linestyle='--',color='gray',zorder=0)
    ax.set_xticks(np.arange(0,len(mtfef_pal)+1,2))
    ax.set_xlim(-1,len(mtfef_pal))
    
ax.set_ylim(0.605,0.653)
    
fig.savefig(fig_folder+'/neuron_dropping.png', dpi=600, bbox_inches='tight')

In [ ]:
df = df2[['sess','outer_fold','neuron_dropped','mean_R2','mean_R2_    FULL']]
df = df.loc[df['sess']!=16]

sessions = df['sess'].unique()

units_sum = pd.DataFrame()
for s in sessions:
    df_sess = df.loc[df['sess']==s].reset_index()
    units_sess = units.loc[units['sess']==str(s)].reset_index()
    
    df_nd = df_sess.groupby(['neuron_dropped','outer_fold']).mean().reset_index()
    df_nd['ci'] = 100*(df_nd['mean_R2_    FULL'] - df_nd['mean_R2'])/df_nd['mean_R2_    FULL']
    df_nd2 = df_nd.groupby(['neuron_dropped']).mean().reset_index()
    units_sess['CI'] = df_nd2['ci'].values
    units_sum = pd.concat([units_sum,units_sess],ignore_index=True)
    
units_sum.head(5)

In [ ]:
from scipy.stats import linregress

def plot_scatter_with_fit(x0,y0,ba,ax):
    brain_areas = np.unique(ba)

    for category in brain_areas:
        mask = ba == category
        x = x0[mask]
        y = y0[mask]
        if category=='FEF':
            col = 'mediumpurple'
            lab = 'FEF'
            ypos = 0.85
        else:
            col = 'mediumseagreen'
            lab = 'MT'
            ypos = 0.73

        ax.scatter(x, y, color=col, label=lab)

        slope, intercept, r_value, p_value, std_err = linregress(x, y)
        ax.plot(x, slope * x + intercept, color=col)

        corr_coef = np.corrcoef(x, y)[0, 1]
        if p_value>0.05:
            p_value_text = f'p = {p_value:.3f}'
        else:
            p_value_text = r"$\mathbf{p = }$" + f" $\mathbf{{{p_value:.3f}}}$"
            
        corr_text = f'r = {corr_coef:.2f}\n{p_value_text}'
        ax.text(0.75, ypos, corr_text, transform=ax.transAxes, color=col)

In [ ]:
plt.rcParams.update({'font.size': 12})

df = units_sum.loc[units_sum['sess']==str(29)]

fig, ax = plt.subplots(1,2,figsize=(10,4))
fig.tight_layout()
plt.subplots_adjust(wspace=0.05, hspace=0.1)

ax = ax.flatten()

plot_scatter_with_fit(df['SNR'].values,df['CI'].values,df['BrainArea'].values,ax[0])
ax[0].set_yticks([0,1,2,3,4])
plot_scatter_with_fit(df['mnFR_bestDir'].values,df['CI'].values,df['BrainArea'].values,ax[1])
ax[1].set_yticks([])
#ax[1].set_xticks([0.1,0.2,0.3,0.4])

fig.savefig(fig_folder+'/neuron_snrVdi.png', dpi=600, bbox_inches='tight')

In [ ]:
from scipy.stats import pearsonr
from scipy.stats import ttest_1samp

snr_fef, di_fef, snr_mt, di_mt = [],[],[],[]
for s in sessions:
    u = units_sum.loc[(units_sum['sess']==str(s)) & (units_sum['BrainArea']=='FEF')]
    if len(u)>0:
        snr, _ = pearsonr(u['SNR'].values,u['CI'].values)
        snr_fef.append(snr)
        di, _ = pearsonr(u['DI'].values,u['CI'].values)
        di_fef.append(di)
    
    u = units_sum.loc[(units_sum['sess']==str(s)) & (units_sum['BrainArea']=='MT')]
    snr, _ = pearsonr(u['SNR'].values,u['CI'].values)
    snr_mt.append(snr)
    di, _ = pearsonr(u['DI'].values,u['CI'].values)
    di_mt.append(di)
    
snr_fef = np.array(snr_fef)
di_fef = np.array(di_fef)
snr_mt = np.array(snr_mt)
di_mt = np.array(di_mt)

snr_all = np.hstack((snr_fef,snr_mt))
di_all = np.hstack((di_fef,di_mt))
    
fig, ax = plt.subplots(1,2,figsize=(9,4))
fig.tight_layout()
plt.subplots_adjust(wspace=0.08, hspace=0.1)

bin_width = 0.09
bins=np.arange(snr_all.min(), snr_all.max() + bin_width, bin_width)

ax[0].hist(snr_fef, bins=bins, edgecolor='black', facecolor='mediumpurple', alpha=0.5, zorder=1)
num_bins = int((snr_mt.max() - snr_mt.min()) / bin_width)
ax[0].hist(snr_mt, bins=bins, edgecolor='black', facecolor='mediumseagreen',alpha=0.5, zorder=2)
ax[0].axvline(x=0, color='gray', linestyle='--',zorder=0)
ax[0].axvline(x=snr_fef.mean(), color='mediumpurple', linestyle='--',zorder=3)
ax[0].axvline(x=snr_mt.mean(), color='mediumseagreen', linestyle='--',zorder=4)

t_statistic, p_value = ttest_1samp(snr_fef, 0)
if p_value>0.05:
    ax[0].text(0.46, 3.8, f'p = {p_value:.3f}', ha='center', color='mediumpurple')
else:
    ax[0].text(0.46, 3.8, r"$\mathbf{p = }$" + f" $\mathbf{{{p_value:.3f}}}$", ha='center', color='mediumpurple')
t_statistic, p_value = ttest_1samp(snr_mt, 0)
if p_value>0.05:
    ax[0].text(0.46, 3.5, f'p = {p_value:.3f}', ha='center', color='mediumseagreen')
else:
    ax[0].text(0.46, 3.5, r"$\mathbf{p = }$" + f" $\mathbf{{{p_value:.3f}}}$", ha='center', color='mediumseagreen')
ax[0].set_yticks([0,1,2,3,4])
ax[0].set_xlim(-0.6,0.85)
ax[0].set_ylim(0,4.5)
ax[0].set_xticks([-0.6,-0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8])

#bin_width = 0.09
bins=np.arange(di_all.min(), di_all.max() + bin_width, bin_width)

ax[1].hist(di_fef, bins=bins, edgecolor='black', facecolor='mediumpurple', alpha=0.5, zorder=1)
ax[1].hist(snr_mt, bins=bins, edgecolor='black', facecolor='mediumseagreen',alpha=0.5, zorder=2)
ax[1].axvline(x=0, color='gray', linestyle='--',zorder=0)
ax[1].axvline(x=di_fef.mean(), color='mediumpurple', linestyle='--',zorder=3)
ax[1].axvline(x=di_mt.mean(), color='mediumseagreen', linestyle='--',zorder=4)

t_statistic, p_value = ttest_1samp(di_fef, 0)
if p_value>0.05:
    ax[1].text(-0.37, 3.8, f'p = {p_value:.3f}', ha='center', color='mediumpurple')
else:
    ax[1].text(-0.37, 3.8, r"$\mathbf{p = }$" + f" $\mathbf{{{p_value:.3f}}}$", ha='center', color='mediumpurple')

t_statistic, p_value = ttest_1samp(di_mt, 0)
if p_value>0.05:
    ax[1].text(-0.37, 3.5, f'p = {p_value:.3f}', ha='center', color='mediumseagreen')
else:
    ax[1].text(-0.37, 3.5, r"$\mathbf{p = }$" + f" $\mathbf{{{p_value:.3f}}}$", ha='center', color='mediumseagreen')

ax[1].set_yticks([])
ax[1].set_xlim(-0.6,0.85)
ax[1].set_ylim(0,4.5)
ax[1].set_xticks([-0.6,-0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8])

fig.savefig(fig_folder+'/neuron_snrVdi_allSess.png', dpi=600, bbox_inches='tight')

# Figure 5

In [3]:
conditions = ["nm10"]
column_names = ['sess','outer_fold','repeat','nMT','nFEF','model','condition','train','test','mean_R2','mean_rho','neuron_inds','time_elapsed']
df = get_outputsSweep(data_folder+'/cross_conditions',conditions,column_names)
df.head(15)

,sess,outer_fold,repeat,nMT,nFEF,model,condition,train,test,mean_R2,mean_rho,neuron_inds,time_elapsed
0,29,0,0,10,10,0,contrast,lo,lo,0.531096,0.746705,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",2.142998
1,29,0,0,10,10,0,contrast,lo,hi,0.471917,0.719708,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.337904
2,29,0,0,10,10,0,contrast,hi,lo,0.514857,0.738644,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.461998
3,29,0,0,10,10,0,contrast,hi,hi,0.479636,0.721723,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.309868
4,29,0,0,10,10,0,contrast,mix,mix,0.486275,0.698971,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.364138
5,29,0,0,10,10,0,speed,lo,lo,0.397906,0.651066,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.314437
6,29,0,0,10,10,0,speed,lo,hi,0.474348,0.766890,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.309768
7,29,0,0,10,10,0,speed,hi,lo,0.147400,0.628794,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.291245
8,29,0,0,10,10,0,speed,hi,hi,0.585386,0.786926,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.303218
9,29,0,0,10,10,0,speed,mix,mix,0.562827,0.750757,"[6, 13, 14, 15, 17, 26, 29, 32, 35, 36, 49, 51...",0.293931
